In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import griddata
from matplotlib import cm

In [2]:
# 让图片中可以显示中文
plt.rcParams['font.sans-serif'] = 'SimHei'
# 让图片中可以显示负号
plt.rcParams['axes.unicode_minus'] = False

In [3]:
# 读取Excel文件，忽略前6行
df0 = pd.read_csv('/home/shuai/视频/20240318.csv', skiprows=6)
df0 = df0.drop(df0.columns[[0,1,3]+list(range(5, 19, 1))+list(range(-1, -5, -1))], axis=1)
display(df0.head())
df0_rows = int(df0.shape[0]/9)
df0_cols = int(df0.shape[1]-2)
df0_values = df0.values

,P2,P4,P7,P8,P9,P10,P11,P12,P13,P14,...,P29,P30,P31,P32,P33,P34,P35,P36,P37,P38
0,1.5,0.5144,7516.780273,41024.984375,13103.160156,104737.664062,12337.788086,98983.453125,11112.787109,89526.882812,...,0,0,374.969391,9960.210938,103.178192,5157.202148,199.192825,6606.042969,1900.335693,18949.912109
1,1.5,1.0290,7186.275391,36246.761719,11951.074219,89518.226562,11195.372070,84728.875000,10027.208008,76764.468750,...,0,0,125.636726,6339.634766,27.254568,4400.720703,60.425423,5197.659180,311.825806,13050.573242
2,1.5,1.5430,8334.772461,45327.628906,10205.735352,88953.109375,9503.373047,84162.937500,8453.971680,76219.468750,...,0,0,44.110538,7327.410156,3.616076,1053.424927,9.274529,1630.518555,11.553834,3542.782715
3,2.0,0.5144,14097.777344,76358.593750,27233.179688,272054.093750,25667.646484,258111.687500,23144.935547,234477.468750,...,0,0,1253.976929,20884.535156,1221.493164,25805.855469,1620.159546,30538.640625,5010.981934,55917.894531
4,2.0,1.0290,13418.339844,80313.000000,26595.847656,258812.203125,25071.730469,244930.578125,22614.595703,221877.000000,...,0,0,1403.122559,24976.859375,904.027893,20220.417969,1234.554077,24038.703125,5224.838867,48107.187500


In [4]:
# 定义插值函数
def interpolate_surface(x, y, z):
    # 创建网格点
    xi = np.linspace(x.min(), x.max(), 100)
    yi = np.linspace(y.min(), y.max(), 100)
    xi, yi = np.meshgrid(xi, yi)
    
    # 进行插值
    zi = griddata((x, y), z, (xi, yi), method='cubic')
    return xi, yi, zi

# 循环处理每个子图
for j in range(df0_cols):
    for i in range(df0_rows):
        x = df0_values[i*9:(i+1)*9, 1]
        y = df0_values[i*9:(i+1)*9, 0]
        z = df0_values[i*9:(i+1)*9, j+2] / 10000
        
        # 插值得到曲面
        xi, yi, zi = interpolate_surface(x, y, z)
        
        # 创建一个三维坐标系
        fig = plt.figure(figsize=(5,5))
        ax = fig.add_subplot(111, projection='3d')

        # 绘制曲面
        ax.plot_surface(xi, yi, zi, cmap=cm.coolwarm)

        jun_zui = '均值' if j%2==0 else '最值'
        # 绘制与zi=50相交的曲面
        # ax.contourf(xi, yi, zi, [50], colors='black', linestyles='solid', linewidths=3)
        if np.any(zi >=49): 
            ax.contourf(xi, yi, zi, levels=[49.9,50.1], colors='black')
        if np.any(zi >=195): 
            ax.contourf(xi, yi, zi, levels=[195.9,196.1], colors='green')
            print(f'{i*45}度缆绳{int(j/2)+1}张力{jun_zui}')

        # 绘制散点图
        ax.scatter(x, y, z)

        # 设置坐标轴标签
        ax.set_xlabel('流速(m/s)')
        ax.set_ylabel('波高(m)')
        ax.set_zlabel('缆绳张力(t)')
        plt.title(f'{i*45}度缆绳{int(j/2)+1}张力{jun_zui}')

        plt.subplots_adjust(left=0.1, right=0.8, top=0.9, bottom=0.1)

        # 保存图形
        plt.savefig(f'{folder_path}/{i*45}度缆绳{int(j/2)+1}张力{jun_zui}.jpg')

        # 关闭图形，以便下一次循环创建新的图形
        plt.close()

In [5]:
# 读取Excel文件，忽略前6行
df1 = pd.read_csv('/home/shuai/视频/20240318.csv', skiprows=6)
df1 = df1.iloc[:, [2, 4]].join(df1.iloc[:, -4:])
display(df1.head())
df1_rows = int(df1.shape[0]/9)
df1_cols = int(df1.shape[1]-2)
df1_values = df1.values
display(df1_rows, df1_cols)

,P2,P4,P39,P40,P41,P42
0,1.5,0.5144,-0.025380,0.017596,76.903023,77.019470
1,1.5,1.0290,-0.011499,0.019110,76.926567,77.064766
2,1.5,1.5430,-0.006904,0.016063,76.961044,77.073494
3,2.0,0.5144,-0.034664,0.046643,76.925583,77.244514
4,2.0,1.0290,-0.033787,0.054192,76.951576,77.203522


5

4

In [6]:
# 定义插值函数
def interpolate_surface(x, y, z):
    # 创建网格点
    xi = np.linspace(x.min(), x.max(), 100)
    yi = np.linspace(y.min(), y.max(), 100)
    xi, yi = np.meshgrid(xi, yi)
    
    # 进行插值
    zi = griddata((x, y), z, (xi, yi), method='cubic')
    return xi, yi, zi

# 循环处理每个子图
for j in range(df1_cols):
    for i in range(df1_rows):
        x = df1_values[i*9:(i+1)*9, 1].reshape(-1)
        y = df1_values[i*9:(i+1)*9, 0].reshape(-1)
        z = df1_values[i*9:(i+1)*9, j+2].reshape(-1)
        
        # 插值得到曲面
        xi, yi, zi = interpolate_surface(x, y, z)
        
        # 创建一个三维坐标系
        fig = plt.figure(figsize=(5,5))
        ax = fig.add_subplot(111, projection='3d')

        # 绘制曲面
        ax.plot_surface(xi, yi, zi, cmap=cm.coolwarm)

        heng_zong = '横荡' if int(j/2)==0 else '纵荡'
        jun_zui = '均值' if j%2==0 else '最值'
        # 绘制与zi=50相交的曲面
        if np.any(zi >=49): 
            ax.contourf(xi, yi, zi, levels=[49.9,50.1], colors='black')
        if np.any(zi >=195): 
            ax.contourf(xi, yi, zi, levels=[195.9,196.1], colors='green')
            print(f'{i*45}度船舶{heng_zong}{jun_zui}')

        # 绘制散点图
        ax.scatter(x, y, z)
        # 设置坐标轴标签
        ax.set_xlabel('流速(m/s)')
        ax.set_ylabel('波高(m)')
        ax.set_zlabel(f'{heng_zong}(m)')
        plt.title(f'{i*45}度船舶{heng_zong}{jun_zui}')

        plt.subplots_adjust(left=0.1, right=0.8, top=0.9, bottom=0.1)

        # 保存图形
        plt.savefig(f'{folder_path}/{i*45}度船舶{heng_zong}{jun_zui}.jpg')

        # 关闭图形，以便下一次循环创建新的图形
        plt.close()